In [53]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import imodelsx.process_results
import sys
sys.path.append('../experiments/')
results_dir = '../results/'
experiment_filename = '../experiments/02_distill_featurized_model.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 387/387 [00:00<00:00, 1515.22it/s]

experiment varied these params: ['dataset_name', 'model_name', 'distiller_name', 'featurizer_name', 'featurizer_frac', 'featurizer_overlap', 'depth', 'pre_interaction', 'gpu']


dataset_name  model_name      distiller_name  featurizer_name  featurizer_frac  featurizer_overlap  depth  pre_interaction  gpu
airfoil       ft_transformer  figs            featurizer       0.33             0                   2      l0l2             0      1
                                                                                                           l1l2             0      1
                                                                                                    3      l0l2             0      1
                                                                                                           l1l2             0      1
                                                                                1                   2      l0l2             0      1
                                                                                                                                  ..
powerplant    resnet          ft_distill      featurizer       0.75       

In [54]:
r = r[r['featurizer_name'] == 'featurizer']

In [69]:
figs = r[r['distiller_name'] == 'figs']
pd.pivot_table(figs, values='distiller_r2_score_test_teacher', aggfunc='median', index='dataset_name', columns=['distiller_name', 'model_name'])

distiller_name           figs          
model_name     ft_transformer    resnet
dataset_name                           
airfoil              0.908227  0.858096
ca_housing           0.957105  0.922546
parkinsons           0.934379  0.618878
powerplant           0.999472  0.992874

In [67]:
pd.pivot_table(figs, values='teacher_r2_score_test_true', aggfunc='median', index='dataset_name', columns=['distiller_name', 'model_name'])

distiller_name           figs          
model_name     ft_transformer    resnet
dataset_name                           
airfoil              0.658617  0.611276
ca_housing           0.623301  0.598641
parkinsons           0.803277  0.550396
powerplant           0.910812  0.910834

In [66]:
ft_distill = r[r['distiller_name'] == 'ft_distill']
pd.pivot_table(ft_distill, values='distiller_r2_score_test_teacher', aggfunc='median', index='dataset_name', columns=['distiller_name', 'model_name'])

distiller_name     ft_distill          
model_name     ft_transformer    resnet
dataset_name                           
airfoil              0.726468  0.702495
ca_housing           0.914319  0.869438
parkinsons           0.656065  0.548019
powerplant           0.995068  0.983545

In [68]:
pd.pivot_table(ft_distill, values='teacher_r2_score_test_true', aggfunc='median', index='dataset_name', columns=['distiller_name', 'model_name'])

distiller_name     ft_distill          
model_name     ft_transformer    resnet
dataset_name                           
airfoil              0.683209  0.638467
ca_housing           0.622120  0.599776
parkinsons           0.806141  0.594082
powerplant           0.907509  0.910817

In [59]:
r.sort_values('teacher_r2_score_test_true', ascending=False)

,dataset_name,subsample_frac,seed,save_dir,model_name,distiller_name,featurizer_name,featurizer_frac,featurizer_overlap,depth,...,distiller_r2_score_train_true,distiller_r2_score_test_true,distiller_r2_score_train_teacher,distiller_r2_score_test_teacher,teacher_f_r2_score_train_true,teacher_f_r2_score_test_true,distiller_f_r2_score_train_true,distiller_f_r2_score_test_true,distiller_f_r2_score_train_teacher,distiller_f_r2_score_test_teacher
330,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,featurizer,0.75,1,3,...,0.930197,0.935241,0.998848,0.998671,0.926873,0.932624,0.926356,0.931789,0.999034,0.998929
110,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,figs,featurizer,0.75,1,3,...,0.932760,0.938877,0.989727,0.989455,0.933595,0.934038,0.930897,0.935135,0.990052,0.989918
371,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,ft_distill,featurizer,0.75,1,3,...,0.928008,0.932509,0.995509,0.995068,0.931150,0.935123,0.927943,0.931942,0.994029,0.993909
147,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,ft_distill,featurizer,0.33,1,3,...,0.925088,0.931247,0.976675,0.976560,0.931001,0.934759,0.923698,0.928150,0.974468,0.974513
359,powerplant,0.2,0,/home/mattyshen/interpretableDistillation/results,resnet,figs,featurizer,0.33,1,3,...,0.931034,0.936178,0.990950,0.989950,0.931078,0.934715,0.929348,0.935725,0.988758,0.988264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,airfoil,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,ft_distill,featurizer,0.75,0,3,...,0.118251,0.022560,0.898647,0.848384,0.236532,0.129828,0.253832,0.196483,0.867792,0.825146
171,airfoil,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,featurizer,0.75,0,2,...,0.118748,-0.133156,0.993431,0.976496,0.200703,0.010317,0.197725,0.001538,0.995106,0.977641
7,airfoil,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,figs,featurizer,0.75,0,3,...,0.099916,-0.091661,0.989191,0.967330,0.113016,-0.090029,0.099503,-0.117191,0.987536,0.951300
190,airfoil,0.2,0,/home/mattyshen/interpretableDistillation/results,ft_transformer,ft_distill,featurizer,0.75,0,3,...,-0.016411,-0.191494,0.935115,0.909161,0.169582,0.024373,0.137506,-0.056828,0.867416,0.755372


In [23]:
# group using these experiment hyperparams when averaging over random seeds
ravg = imodelsx.process_results.average_over_seeds(
    r, experiment_filename, key_to_average_over='seed'
)

# apply cross validation
ravg_cv = (
    ravg
    .sort_values(by='distiller_r2_score_test_true', ascending=False)
    .groupby(by=['model_name', 'dataset_name'])
    .max()
    .reset_index()
)
ravg_cv

,model_name,dataset_name,subsample_frac,save_dir,distiller_name,featurizer_name,featurizer_frac,featurizer_overlap,depth,bit,...,teacher_f_r2_score_test_true,teacher_f_r2_score_test_true_err,distiller_f_r2_score_train_true,distiller_f_r2_score_train_true_err,distiller_f_r2_score_test_true,distiller_f_r2_score_test_true_err,distiller_f_r2_score_train_teacher,distiller_f_r2_score_train_teacher_err,distiller_f_r2_score_test_teacher,distiller_f_r2_score_test_teacher_err
0,ft_transformer,airfoil,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.775495,0.0,0.701852,0.0,0.708028,0.0,0.995520,0.0,0.979945,0.0
1,ft_transformer,ca_housing,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.698138,0.0,0.714189,0.0,0.689043,0.0,0.981429,0.0,0.978224,0.0
2,ft_transformer,parkinsons,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.844082,0.0,0.870817,0.0,0.852187,0.0,0.988125,0.0,0.972679,0.0
3,ft_transformer,powerplant,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.935123,0.0,0.928359,0.0,0.933189,0.0,0.999927,0.0,0.999883,0.0
4,resnet,airfoil,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.714306,0.0,0.688611,0.0,0.708374,0.0,0.950779,0.0,0.893118,0.0
5,resnet,ca_housing,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.684789,0.0,0.715542,0.0,0.691931,0.0,0.943306,0.0,0.939869,0.0
6,resnet,parkinsons,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.750428,0.0,0.864377,0.0,0.838808,0.0,0.927351,0.0,0.878672,0.0
7,resnet,powerplant,0.2,/home/mattyshen/interpretableDistillation/results,ft_distill,no_featurizer,0.75,1,3,1,...,0.935637,0.0,0.931905,0.0,0.935743,0.0,0.998440,0.0,0.998384,0.0


In [5]:
# , hue='dataset_name')
sns.barplot(data=ravg_cv, x='model_name', y='accuracy_test')
plt.show()

ValueError: Could not interpret input 'accuracy_test'

Load an individual model

In [ ]:
run_args = r.iloc[0]
model = joblib.load(join(run_args.save_dir_unique, 'model.pkl'))
model